In [62]:
#use different method to find a certain eigenvalue and eigenvector pair
#(a)Householder orthogonal + upper-triangular(QR)
#(b)Gram-schmidt (QR)
#(c)Householder find symmetric tridiagonal matrix similar to A
#(d)Jacobi's method
#(e)QR method for (c) cont.
#(f)single shift QR method
#(g)variant single shift QR method


#usage = for biweekly quiz 8 for class PMS.CM Chang @ NCTU AM 11
#Biweekly quiz 8 power method and its variants
#Pactice of Mathematics Software

#author = maxwill lin =  yt lin
#school number = 0712238@NCTU

#created 2019.12.27
#modified 

#show ans by printing out

In [63]:
import numpy as np
import scipy
import scipy.linalg
 
np.set_printoptions(suppress=True)

In [64]:
A = np.eye(12)*4
for i in range(0,3):
    A[i][i+1] = A[i+1][i] = -1
for i in range(0,3):
    A[11-i][10-i] = A[10-i][11-i] = -1
for i in range(0,4):
    A[5+i][3+i] = A[3+i][5+i] = -1 
A[4][0] = A[0][4] = -1
A[7][11] = A[11][7] = -1
print(A)
b = np.asarray([220, 110, 110, 220, 110, 110, 110, 110, 220, 110, 110, 220]).T

eps = 1e-5
x0 = np.zeros(A.shape[0])

[[ 4. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  4. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  4. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  4.  0. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  4.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  4.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  4.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  4.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  4. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  4. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  4. -1.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0. -1.  4.]]


In [65]:
def QR_GramSchmidt(A): #for n*n matrix
    n = A.shape[0]
    Q =  np.zeros_like(A) #ortho
    R =  np.zeros_like(A) #R
    for i in range(n):
        v = A[:,i]
        for j in range(i):
            R[j,i] =  np.dot(Q[:,j].T , A[:,i]) 
            v = v - (R[j,i] * Q[:,j])
        R[i,i] =  np.linalg.norm(v)
        Q[:,i] = (v / R[i,i])
    return Q, R


In [66]:
QR_GramSchmidt(A)

(array([[ 0.94280904,  0.20464687,  0.0493541 ,  0.01200232,  0.24868286,
          0.00462621,  0.06664108,  0.00127812,  0.01785665,  0.00478467,
          0.00119308,  0.00068883],
        [-0.23570226,  0.93552855,  0.24471407,  0.06509233,  0.06650852,
          0.01781426,  0.01785318,  0.00478363,  0.0047846 ,  0.00128203,
         -0.00000001,  0.00137765],
        [ 0.        , -0.26311741,  0.92950217,  0.24836698,  0.0173512 ,
          0.06663082,  0.00477166,  0.01785639,  0.00128174,  0.00034343,
         -0.00119311,  0.00482178],
        [ 0.        ,  0.        , -0.2673347 ,  0.92837561,  0.00289627,
          0.24870903,  0.00123345,  0.06664192,  0.00034235,  0.0000917 ,
         -0.00477243,  0.01790947],
        [-0.23570226, -0.11694107, -0.04729768, -0.01708305,  0.92822294,
          0.00069059,  0.24871113,  0.00032886,  0.06664199,  0.01785667,
          0.00477233,  0.00137765],
        [ 0.        ,  0.        ,  0.        , -0.26787995, -0.00576611,
      

In [67]:
""" another style
def QR_Householder(A):
    n, n = A.shape
    Q = np.eye(n) #ortho
    R = A.copy() #trasfromed

    for j in range(n):
        x = R[j:, j]
        normx = np.linalg.norm(x)
        rho = -np.sign(x[0])
        u1 = x[0] - rho * normx
        u = x / u1
        u[0] = 1
        beta = -rho * u1 / normx
        R[j:, :] = R[j:, :] - beta * np.outer(u, u).dot(R[j:, :])
        Q[:, j:] = Q[:, j:] - beta * Q[:, j:].dot(np.outer(u, u))
        
    return Q, R
"""
def householder_transformation(v):
    n = v.shape[1]
    e1 = np.zeros_like(v)
    e1[0, 0] = 1
    v2 = np.linalg.norm(v) * e1
    if v[0, 0] < 0:
        v2 = - v2
    u = v + v2
    H = np.identity(n) - (2*u.T@u)/(u@u.T)
    return H

def QRstep(q, r, i, n):
    v = r[i:, i].reshape(1, -1)
    H = np.identity(n)
    H[i:, i:] = householder_transformation(v)
    r = H@r
    q = q@H
    return q, r

def QRHouseholder(A):
    n = A.shape[0]
    Q = np.identity(n)
    R = A.copy()
    for i in range(n):
        Q, R = QRstep(Q, R, i, n)
    R = R[:n, :n]
    return Q, R

In [70]:
QRHouseholder(A)

(array([[-0.94280904, -0.20464687, -0.0493541 , -0.01200232, -0.24868286,
         -0.00462621, -0.06664108, -0.00127812, -0.01785665, -0.00478467,
         -0.00119308, -0.00068883],
        [ 0.23570226, -0.93552855, -0.24471407, -0.06509233, -0.06650852,
         -0.01781426, -0.01785318, -0.00478363, -0.0047846 , -0.00128203,
          0.00000001, -0.00137765],
        [ 0.        ,  0.26311741, -0.92950217, -0.24836698, -0.0173512 ,
         -0.06663082, -0.00477166, -0.01785639, -0.00128174, -0.00034343,
          0.00119311, -0.00482178],
        [ 0.        ,  0.        ,  0.2673347 , -0.92837561, -0.00289627,
         -0.24870903, -0.00123345, -0.06664192, -0.00034235, -0.0000917 ,
          0.00477243, -0.01790947],
        [ 0.23570226,  0.11694107,  0.04729768,  0.01708305, -0.92822294,
         -0.00069059, -0.24871113, -0.00032886, -0.06664199, -0.01785667,
         -0.00477233, -0.00137765],
        [ 0.        ,  0.        ,  0.        ,  0.26787995,  0.00576611,
      

In [69]:
def outer_norm(A):
    lt = np.tril_indices(A.shape[0], -1)
    ut = np.triu_indices(A.shape[0], 1)
    return np.sum(np.square(A[lt])) + np.sum(np.square(A[ut]))

def outer_argmax(A):
    B = np.abs(A.copy())
    B[np.diag_indices(B.shape[0])] = 0
    return np.unravel_index(B.argmax(), B.shape)

def sign(x):
    return x/abs(x) if x != 0 else 0

def jacobi_method(A):
    ls = []
    norm = outer_norm(A)
    ls.append(norm)
    for _ in range(A.shape[0]-1):
        i, j = outer_argmax(A)
        if i != 0 or j != 0:
            a = A.copy()
            theta = 0.5 * (A[i, i] - A[j, j]) / A[i, j]
            t = sign(theta) / (abs(theta) + np.sqrt(theta**2 + 1))
            c = 1 / np.sqrt(t**2 + 1)
            s = c * t
            a[i, j] = a[j, i] = 0
            a[i, i] = A[i, i] + t * A[i, j]
            a[j, j] = A[j, j] - t * A[i, j]
            for l in range(A.shape[0]):
                if l != i and l != j:
                    a[i, l] = a[l, i] = c * A[i, l] + s * A[j, l]
                    a[j, l] = a[l, j] = c * A[j, l] - s * A[i, l]
            norm -= 2 * A[i, j]**2
            A[:] = a
            ls.append(norm)
        else:
            ls.append(0.0)
    return ls

In [56]:
jacobi_method(A)

[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]